**Coursebook: Using LLM for Web Scraping and Prompt Design**

- Part 4 of using LLM for web scraping and prompt design
- Course Length: 9 hours
- Last Updated: July 2023
---

Developed by Algoritma's Research and Development division

## Background

The coursebook is part of the **Large Language Models Specialization** developed by [Algoritma](https://algorit.ma/). The coursebook is intended for a restricted audience only, i.e. the individuals and organizations having received this coursebook directly from the training organization. It may not be reproduced, distributed, translated or adapted in any form outside these individuals and organizations without permission.Algoritma is a data science education center based in Jakarta. We organize workshops and training programs to help working professionals and students gain mastery in various data science sub-fields: data visualization, machine learning, data modeling, statistical inference etc.

# Using LLM for Web Scraping and Prompt Design

In this module, we will explore the utilization of Large Language Models (LLM) for Web Scraping and Information Extraction. The main objectives are as follows:

- **Leveraging Large Language Models (LLM) for Web Scraping and Information Extraction**
   - Using LLM for web scraping
   - Introduction to the steps involved in connecting website URLs with LLM
   - Introduction to LlamaIndex and its usage in web scraping
   - Demonstration of using LangChain and OpenAI to build a Question-Answering System with website data
   
- **Optimizing Large Language Models (LLM) for Performance Enhancement**
   - Demonstration of using LlamaIndex
   - Designing effective prompts for LLM
   - Using LangChain's Caching to enhance LLM performance
   - Demonstration of prompt design and caching to maximize LLM usage

- **Ethical Considerations and Future Implications of Generative AI with Large Language Models (LLM)**
   - A language for LLM prompt design: Guidance
   - Understanding the ethical considerations of Generative AI
   - Impact on privacy, bias, and misinformation
   - Responsible user of Large Language Models in society
   - Discussion on the future of Generative AI and its potential impact

## Leveraging Large Language Models (LLM) for Web Scraping and Information Extraction

In this digital era, websites like Nest have become vital sources of information exchange. Many companies utilize the web to showcase their services, and users often seek valuable information from these websites. However, summarizing the data manually can be time-consuming. By leveraging Large Language Models (LLM), such as our LLM-powered solution, it becomes much easier to automate the process of extracting and summarizing information from websites. Our LLM-based system can efficiently process and analyze web content, allowing users to access relevant and concise information with increased speed and accuracy.

LangChain is one of the tools that can be used to leverage LLM for web scraping and information extraction. With LangChain, you can easily build web scraping applications that extract relevant and useful information from websites, saving time and effort in manual data extraction.

### Using LangChain to Scrape Information from Website

In addition to using text and PDF documents by loading them as `Document` using `Loader`, we can also extract information from websites and feed it to Large Language Models (LLM) using the `WebBaseLoader` from `langchain`. 

The `WebBaseLoader` enables us to scrape data and content from web pages and utilize LLM to process and analyze the extracted information. This provides us with a versatile and comprehensive approach to gather and understand data from various sources, making it easier to perform tasks such as question-answering, summarization, and more, by leveraging the power of LLM in web scraping and information extraction.

[`WebBaseLoader` documentation](https://python.langchain.com/docs/integrations/document_loaders/web_base)

In [44]:
from langchain.document_loaders import WebBaseLoader

# load FAQ blog about Algoritma
loader = WebBaseLoader("https://blog.algorit.ma/faq-bootcamp-algoritma-data-science/")
docs = loader.load()

`docs` is a list of documents, the component of each document are:
- `page_content`: The content/information of page
- `metadata`: The detail information about the page content, like source, language and title of the page

In [48]:
# Only got 1 document because we only scrape from 1 website source
len(docs)

1

In [50]:
# see the metadata
docs[0].metadata

{'source': 'https://blog.algorit.ma/faq-bootcamp-algoritma-data-science/',
 'title': 'FAQ Bootcamp Algoritma Data Science School',
 'description': 'Temukan jawaban dari beberapa pertanyaan yang sering ditanyakan seputar Bootcamp Data Science di Algoritma Data Science School.',
 'language': 'id'}

In [51]:
# the length of the page_content
len(docs[0].page_content)

10053

In [49]:
# see the first of 100 character page_content
docs[0].page_content[:100]

'\n\n\n\n\nFAQ Bootcamp Algoritma Data Science School\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

Notice there are so many "\n\n" indicating newlines and line breaks. Before embedding this text, it is necessary to clean it and divide the text into smaller chunks using `CharacterTextSplitter`. 

This preprocessing step is important as it helps to remove unwanted characters, spaces, and formatting, making the text suitable for embedding and subsequent natural language processing tasks. By splitting the text into smaller chunks, we can efficiently process and analyze the content to build a powerful question-answering system or perform other text-related tasks using Large Language Models (LLM).

In [53]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

algo_texts = text_splitter.split_documents(docs)

Created a chunk of size 199, which is longer than the specified 100
Created a chunk of size 7513, which is longer than the specified 100
Created a chunk of size 236, which is longer than the specified 100
Created a chunk of size 179, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 134, which is longer than the specified 100
Created a chunk of size 226, which is longer than the specified 100


In [54]:
# see how many chunks created
len(algo_texts)

22

In [55]:
# See the first chunk
# Even tho there are several \n left, but so many \n removed compared with docs[0].page_content[:100]
algo_texts[0]

Document(page_content='FAQ Bootcamp Algoritma Data Science School\n\n\n \n\n\nSkip to content\n\nAbout\n\n \nSearch', metadata={'source': 'https://blog.algorit.ma/faq-bootcamp-algoritma-data-science/', 'title': 'FAQ Bootcamp Algoritma Data Science School', 'description': 'Temukan jawaban dari beberapa pertanyaan yang sering ditanyakan seputar Bootcamp Data Science di Algoritma Data Science School.', 'language': 'id'})

After dividing the text into smaller chunks using the `CharacterTextSplitter`, we can proceed to embed these chunks of text using the `SentenceTransformerEmbeddings`. The resulting embeddings are then stored in Chroma, which serves as a vector database, enabling us to efficiently represent and manage the text data.


In [56]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma


Once the text has been embedded and stored in Chroma, we can create a QnAChain similar to what we did in Module 3. This QnAChain allows us to perform question-answering tasks using the information stored in Chroma. By leveraging the embeddings and Chroma database, we can build a robust question-answering system that is capable of handling large text datasets obtained from web scraping and information extraction tasks. This approach enhances the efficiency and effectiveness of our QnA system by using advanced embedding techniques and a dedicated vector database to manage the text data.

In [57]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Name the chroma collection as "algo-faq"
algo_db = Chroma.from_documents(algo_texts, embedding_function, collection_name="algo-faq")

After creating the `SentenceTransformerEmbeddings` as an open-source embedding function and initializing the Chroma collection named `"algo-faq"`, the next step is to create the Large Language Model (LLM) using OpenAI. This is achieved by importing the necessary modules and loading the OpenAI token from the environment using the `load_dotenv()` function.

In [ ]:
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain import OpenAI

# Load OpenAI token from env
load_dotenv()
# create llm using OpenAI
llm = OpenAI()

Once the OpenAI token is loaded, we can proceed to create the QnA Chain. The QnA Chain is established using the `RetrievalQA` class from the LangChain library. The QnA Chain is built by specifying the LLM model (llm) obtained from OpenAI and setting the `chain_type` as `"stuff"`. Additionally, the retriever is defined using the previously created Chroma collection (`algo_db`) as the data source for searching relevant documents or chunks related to the input questions.

In [58]:
# Create the QnA Chain
algo_faq = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=algo_db.as_retriever()
)

With this QnA Chain in place, we can efficiently perform question-answering tasks using the Large Language Model to retrieve relevant information from the Chroma database. This allows us to build an effective QnA system that leverages advanced embedding techniques and the power of Large Language Models to provide accurate and contextually appropriate answers to user queries based on the data stored in the Chroma collection.

In this scenario, we have developed a bot/QnA system that enables users to ask questions in multiple languages and receive contextually relevant answers based on company QnA information. The QnA system is specifically designed for the Algoritma company. Users can interact with the bot by asking questions related to various topics, and the system utilizes the power of Large Language Models and advanced embedding techniques to find the most appropriate answers from the Algoritma QnA database. The system's capability to handle multilingual questions and provide contextually accurate responses makes it a valuable tool for efficient and effective information retrieval.

In [42]:
# Asking qustion
algo_faq("Why study in Algoritma Data Science?")

{'query': 'Why study in Algoritma Data Science?',
 'result': ' Algoritma Data Science School provides a comprehensive curriculum for students to learn data science, with benefits such as lifetime learning, full course material, portfolio projects, teaching assistant, mentoring sessions, career preparation services, and quality facilities.'}

In [43]:
algo_faq("Metode belajar di Algoritma?")

{'query': 'Metode belajar di Algoritma?',
 'result': ' Metode belajar di Algoritma Data Science School terbagi dua, yaitu Onsite dan Online. Onsite adalah Belajar dengan metode offline di kelas Algoritma yang terletak di Menara Menara Standard Chartered, Lantai 31. Online adalah belajar dari mana saja Anda mau dengan metode online interaktif.'}

In [59]:
algo_faq("Apakah program di Algoritma dapat diikuti orang awam?")

{'query': 'Apakah program di Algoritma dapat diikuti orang awam?',
 'result': ' Ya, program di Algoritma Data Science School dapat diikuti oleh orang awam tanpa latar belakang pendidikan IT sebelumnya. Contohnya seperti Reyna Cheryl Sondakh dan Samuel Gema yang telah berhasil menyelesaikan program dengan background non-IT dan kini telah menjadi data scientist atau data analyst di perusahaan ternama di Indonesia.'}

From the output results, we can see that the model can provide answers based on the questions given. It's fascinating because we can ask questions in any language, even though the web database may not contain information in various languages. This is possible because the Large Language Model (LLM) used in this Q&A system has been pretrained on vast amounts of multilingual data, enabling it to understand and respond to queries in different languages. The model's ability to handle multilingual inputs makes it versatile and useful for users worldwide, regardless of their preferred language.

### Using LLama Index to Create QnA Web-based information

We can also use another tool called **LLama Index** to create a Q&A system for web-based information. The LLama Index differs from LangChain in that it focuses on enhancing document management through advanced technology, offering an intuitive and efficient way to search and summarize documents using Large Language Models (LLMs) and **innovative indexing techniques**. The LLama Index aims to provide a comprehensive solution for handling and processing large volumes of text-based data, making it easier for users to extract valuable insights from web content and other unstructured documents.

LLama Index is more geared towards document management and information retrieval from **web-based sources**. It is useful for scenarios where efficient searching and summarization of large volumes of unstructured text data are required. The use cases for LLama Index include web scraping and indexing, summarizing web-based content, and organizing and retrieving information from large text databases.


<!--[source1](https://medium.com/badal-io/exploring-langchain-and-llamaindex-to-achieve-standardization-and-interoperability-in-large-2b5f3fabc360) [source2](https://gpt-index.readthedocs.io/en/latest/index.html#why-llamaindex).-->



We can use `TrafilaturaWebReader` from LLama Index to easily scrape and extract information from web pages. `TrafilaturaWebReader` is a component of LLama Index that is specialized in web scraping. It allows us to fetch the contents of web pages, process the HTML documents, and extract relevant information such as text, metadata, and links.

By utilizing `TrafilaturaWebReader`, we can efficiently gather data from various web sources and use the extracted information for further analysis, summarization, or integration with other applications. It simplifies the process of web scraping and makes it more convenient to work with unstructured web-based data.

For example, if we want to create a Q&A system that gathers information from multiple websites and provides answers based on user queries, we can use `TrafilaturaWebReader` to collect data from the web, process the web pages, and extract relevant content. This can be a valuable tool in various applications, such as content aggregation, market research, and data analysis.

In [61]:
# Using Trafilatura to Scrape information from web
from llama_index import TrafilaturaWebReader

In [63]:
documents = TrafilaturaWebReader().load_data(["https://blog.algorit.ma/faq-bootcamp-algoritma-data-science/"])

len(documents)

1

In [64]:
len(documents)

1

In [65]:
documents

[Document(id_='21fbf7e9-3972-4eda-bb17-7cb62785682a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='69b226edcf37ea64cfc32e7e9a0eba2c4bf7680cb0f5d00d90eba8757072e169', text='Frequently Asked Questions (FAQ) Bootcamp Algoritma Data Science School\nTemukan jawaban dari beberapa pertanyaan yang sering ditanyakan seputar Bootcamp Data Science di Algoritma Data Science School.\nTable of Contents\nKenapa Belajar di Algoritma Data Science School?\n- Lifetime Learning\nSeluruh Alumni Algoritma (Academy Regular dan Full-Stack) mendapatkan akses mengikuti workshop yang diadakan oleh Tim Algoritma secara gratis seumur hidup!\n- Materi Lengkap\nMateri Kursus (PDF & HTML), dataset untuk latihan, catatan referensi, dan worksheet (Notebook R atau Notebook Jupyter) dapat diakses dengan mudah melalui akun Learning Management System.\n- Portfolio Project\nProyek Data Science yang Anda buat sesuai dengan perkembangan industri dan kasus

In [68]:
# see the first 200 text content
# more clean than langchain webbaseloader
documents[0].text[:200]

'Frequently Asked Questions (FAQ) Bootcamp Algoritma Data Science School\nTemukan jawaban dari beberapa pertanyaan yang sering ditanyakan seputar Bootcamp Data Science di Algoritma Data Science School.\n'

After gathering and processing the web-based data using `TrafilaturaWebReader`, we create a Vector Index, which is a data structure designed for efficient retrieval of items based on keys, in this case, the keys are information vectors stored in the `Chroma` vector database.

The Vector Index, implemented as `VectorStoreIndex`, is a key component of LLama Index that optimizes the retrieval process for specific vectors stored in `Chroma`. By organizing the data in this structured manner, it significantly improves the speed and efficiency of searching and retrieving relevant information.

With the help of Vector Index, we can quickly access and retrieve specific information vectors from the `Chroma` vector database without having to search through the entire database. This enhances the performance of our application and makes it more suitable for real-time or large-scale information retrieval tasks.

[VectorStoreIndex](https://gpt-index.readthedocs.io/en/latest/core_modules/data_modules/index/index_guide.html#vector-store-index)

In [74]:
from llama_index import GPTVectorStoreIndex
import chromadb

The step we need to take is to create an empty collection to store our data in the LLama Index. A collection is a logical grouping of documents or pieces of information that are related to a specific topic or domain.



In [75]:
# Create empty collection
chroma_client = chromadb.Client()
algo_collection = chroma_client.create_collection('algo-faq')

In the context of LLama Index, creating an empty collection involves **setting up a space to store the documents or information vectors that we will extract and process from various web sources**. This collection acts as a container to hold all the relevant data related to a particular subject.

Then, we create the Vector Index and store it in the 'index' object using the `GPTVectorStoreIndex`. To do this, we need to pass the `documents` and the `chroma_collection` as parameters to the `GPTVectorStoreIndex` constructor. 

This index will efficiently organize and manage the information vectors, allowing us to quickly retrieve and access the relevant data for various tasks such as question-answering systems or information retrieval.

In [79]:
# Load OpenAI token from env
load_dotenv()
# Create Vector Index and store it to 'index' object
index = GPTVectorStoreIndex.from_documents(documents, 
                                           chroma_collection = algo_collection)


We need to create a query engine from the index. Essentially, the query engine is used for efficiently searching and retrieving relevant information from the index based on user queries or input. It allows us to perform searches and find answers to questions in an efficient and accurate manner by leveraging the underlying data structure and indexing techniques.

In [ ]:
# Create query engine from index
query_engine = index.as_query_engine()

In [80]:
# Query/ask question from query_engine
response = query_engine.query("Apakah program di Algoritma dapat diikuti orang awam?")

response

Response(response='\nYa, program di Algoritma dapat diikuti oleh orang awam. Algoritma tidak membatasi usia untuk belajar data science bersama Algoritma, umumnya yang mengikuti program ini berusia 17 - 55 tahun.', source_nodes=[NodeWithScore(node=TextNode(id_='a8b1ac9d-bad1-491a-b233-9a9169d2deb4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='21fbf7e9-3972-4eda-bb17-7cb62785682a', node_type=None, metadata={}, hash='69b226edcf37ea64cfc32e7e9a0eba2c4bf7680cb0f5d00d90eba8757072e169'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='8dd47171-f2a9-4aab-b9d4-c1701a86f662', node_type=None, metadata={}, hash='c5a3f1870b9d87f2ed3ec9d5f246d4d51f912288ec31cd9978caed1cda07a20f')}, hash='8c3a398efe18675002915bebedb4637136a3b312e200da01f35699ac2b835206', text='hingga meraih mimpi yang Anda inginkan. Sehingga Algoritma pun tidak membatasi usia untuk belajar data science b

In [82]:
print(f"Answer: {response}")

Answer: 
Ya, program di Algoritma dapat diikuti oleh orang awam. Algoritma tidak membatasi usia untuk belajar data science bersama Algoritma, umumnya yang mengikuti program ini berusia 17 - 55 tahun.


### Wrap Together Into Function

To simplify the code and make it easier to apply repeatedly, we can wrap it into a function. This way, we can call the function with different inputs or parameters, and it will perform the desired operations without the need to write the same code repeatedly. Function encapsulation makes the code more organized and reusable, enhancing code readability and maintainability.

In [83]:
# Create embedding
def create_embedding_store(name):
    chroma_client = chromadb.Client()
    return chroma_client.create_collection(name)

# Query Pages and get list of questions
def query_pages(collection, urls, questions):
    # Get content from webpages
    docs = TrafilaturaWebReader().load_data(urls)
    # create vectorstoreindex
    index = GPTVectorStoreIndex.from_documents(docs, chroma_collection=collection)
    # Create query engine
    query_engine = index.as_query_engine()
    # Iterate, query and answer every questions
    for question in questions:
        print(f"Question: {question}")
        print(f"Answer: {query_engine.query(question)} \n\n")

The provided function simplifies the process of building a Q&A system by encapsulating the steps of creating an embedding store, querying web pages, and generating answers into a single function. It makes it easier for users to provide web page URLs and questions, and get relevant answers from the system.

> **Example**: After we create the function, we can use it to analyze web pages or online reviews if we have a product and want to know more about the characteristics and opinions of the reviewers who are using our product. This Q&A system will allow us to input relevant questions related to our product, and it will extract information from the web pages or reviews to provide us with valuable insights and feedback from the users. This can help us understand the strengths and weaknesses of our product and identify areas for improvement to better meet the needs and preferences of our customers.

In [ ]:
# Get product review for 2 pages
url_list_erha = ["https://reviews.femaledaily.com/products/cleanser/facial-wash/erha/erha-1-facial-wash?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=honest&page=1",
                 "https://reviews.femaledaily.com/products/cleanser/facial-wash/erha/erha-1-facial-wash?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=honest&page=2"]

questions_erha = [
    "How long the reviewer use Erha 1?",
    "What is the sentiment review of this product and what its about?",
    "Is there any suggested improvement from reviewer?",
    "What is the reviewer's favorite thing about this product?" 
]

collection_erha = create_embedding_store("erha1")

In [17]:
# query the question and generate answer
query_pages(
    collection_erha,
    url_list_erha,
    questions_erha
)

Question: How long the reviewer use Erha 1?
Answer: 
More than 1 year 


Question: What is the sentiment review of this product and what its about?
Answer: 
The sentiment review of this product is generally positive. Customers have found that the product is gentle on the skin, has a clear texture, and has a faint chemical smell. It lathers well and leaves the skin feeling soft and not dry. The packaging is compact and travel friendly, and the price is affordable. Customers have also found that the product lasts a long time and is easy to repurchase. 


Question: Is there any suggested improvement from reviewer?
Answer: 
Yes, there are suggested improvements from the reviewers. For example, Zahradestriana recommends using a facial wash with AHA, DMAE, and Aloe Vera Extract for normal and dry skin. Novitawd suggests finding a facial wash that is free of SLS. Windiiw_ recommends using a facial wash with Niacinamide, AHA, DMAE, and Aloe Vera Extract for anti-aging and moisturizing benefits

## Optimizing Large Language Models (LLM)

The main problem with LLMs lies in their computational and memory requirements. These models are very large and complex, often requiring extensive computational resources and high memory capacity to run efficiently. This makes them challenging to deploy and use in resource-constrained environments, such as on edge devices or in production systems with limited hardware capabilities.

To address these issues, researchers and developers are continuously working on **optimizing LLMs**. This optimization process includes techniques like model **caching**. 
The caching technique in Langchain helps to improve the efficiency and speed of processing, especially when dealing with large datasets or complex text documents. By caching embeddings, the system can avoid redundant computations, which can be time-consuming and resource-intensive.

By optimizing LLMs, we can make these powerful language models more accessible and applicable to a wider range of applications, addressing the computational, memory, and efficiency challenges they pose.

### LangChain Caching

When working with Large Language Models (LLMs), such as GPT-3, it is crucial to monitor token usage and associated costs. LLMs charge users based on the number of tokens used in each API call, and these tokens correspond to the individual components of the input text, including words and characters. LangChain offers an optional [caching layer](https://python.langchain.com/docs/modules/model_io/models/llms/llm_caching) for Large Language Models (LLMs), which provides significant benefits in two main aspects:

1. **Cost Optimization:** Monitoring token usage allows us to keep track of API credit consumption and manage our budget effectively. It helps us avoid overage charges and stay within our desired usage limits.

2. **Model Selection:** By understanding the token count for different queries, we can choose the most suitable LLM model based on its token limit. This ensures that our queries fit within the model's constraints and prevents unnecessary additional costs.

3. **Performance Optimization:** Tracking token usage also helps us optimize the size and complexity of queries. By managing token usage efficiently, we can improve the overall performance and responsiveness of our application.


By employing the caching feature, LangChain aims to improve the efficiency and user experience of applications that heavily rely on LLMs. It provides a cost-effective and time-saving solution by intelligently managing the retrieval and storage of completion results, making it a valuable tool for optimizing LLM-based applications.

First, let's keep track of the number of tokens used and the cost incurred for our queries. We can achieve this by utilizing the [`get_openai_callback()`](https://python.langchain.com/docs/modules/model_io/models/llms/token_usage_tracking) function.


In [85]:
from langchain.callbacks import get_openai_callback

We will create a question-answering system using `algo_faq`, and track the token usage and cost with `get_openai_callback()`. The purpose of using `get_openai_callback()` is to monitor the resource consumption during the query, helping us manage costs and make informed decisions when utilizing the API.

In [91]:
with get_openai_callback() as cb:
    result = algo_faq("Apakah program di Algoritma dapat diikuti orang awam?")
    print(f"Question: {result['query']}\nAnswer:\n{result['result']} \n")
    print("---- cb -----")
    print(cb)

Question: Apakah program di Algoritma dapat diikuti orang awam?
Answer:
 Ya, program di Algoritma dapat diikuti orang awam yang tidak memiliki latar belakang pendidikan IT sebelumnya. Algoritma akan mengajarkan dasar-dasar data science, seperti dasar-dasar pemrograman dan statistik praktis, sehingga program ini terbuka untuk siapa saja yang tidak memiliki latar belakang IT. Contohnya seperti Reyna Cheryl Sondakh dan Samuel Gema. 

---- cb -----
Tokens Used: 2910
	Prompt Tokens: 2786
	Completion Tokens: 124
Successful Requests: 1
Total Cost (USD): $0.0582


The output shows the information collected by the `get_openai_callback()` during the execution of the query. Here's the breakdown:

- **Tokens Used**: The total number of tokens consumed by the query, including both the prompt tokens and the completion tokens.
- **Prompt Tokens**: The number of tokens used for the prompt, which is the initial part of the input text before the model generates the completion.
- **Completion Tokens**: The number of tokens used for the model's completion, which is the generated answer or response to the prompt.
- **Successful Requests**: The number of successful requests made to the OpenAI API. In this case, there was one successful request to obtain the answer.
- **Total Cost (USD)**: The total cost incurred for making the API request, which is calculated based on the number of tokens used. In this example, the cost is $0.0582.

This information is useful for understanding the resource utilization and costs associated with running the query with the large language model. It helps in optimizing the usage and managing the budget when utilizing the OpenAI API for language processing tasks.

Caching in LangChain is a technique that helps to reduce costs for repetitive queries by storing the results of previous queries and reusing them when the same query is made again. It improves the efficiency of the application and reduces the need for making redundant API calls to the language model provider.

There are two ways to implement caching in LangChain:

1. **In Memory Cache**: In this approach, the results of previous queries are stored in memory, typically using data structures like dictionaries or hash maps. This allows for fast and efficient access to cached results, as the data is stored directly in the application's memory. However, the downside is that the cached data is lost when the application is restarted.

2. **SQLite Cache**: This approach involves using a lightweight, file-based database system like SQLite to store the cached results. SQLite provides a persistent storage solution, which means the cached data is preserved even if the application is restarted. This makes it a more durable caching method compared to the in-memory cache.

By using caching, LangChain can quickly retrieve the results of previously processed queries, reducing the need to make additional expensive API calls. This not only helps in cost optimization but also enhances the overall performance of the application, providing a more responsive and efficient experience for users. It's important to choose the appropriate caching method based on the specific requirements of the application and the desired trade-offs between memory usage and persistence of cached data.

#### In Memory Cache

In-memory cache is like having a handy notepad where us jot down important information so we can quickly access it later without having to look it up again. In the context of LangChain, it works similarly. When we make a query using the language model, the results are temporarily stored in the computer's memory, just like writing them down on the notepad.

The next time we make the same query, instead of going through the whole process again, LangChain checks the in-memory cache first. If it finds the results from the previous query, it can quickly retrieve them, saving time and resources. This way, repetitive queries become faster because LangChain doesn't need to do the same work again and again. However, it's essential to remember that in-memory cache is temporary and may get cleared when we close the program or restart your computer, just like how your notepad gets erased when we close it.

To utilize the in-memory cache in LangChain, we can use the `InMemoryCache` by defining it in `langchain.llm_cache`. This caching mechanism allows LangChain to store and retrieve results from previous queries in the computer's memory, making repetitive queries faster and more efficient. By implementing the `InMemoryCache`, LangChain can quickly access previously computed results without needing to recompute them, leading to cost savings and improved performance.

In [92]:
# In memory cache
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [93]:
# Let's see caching impact
with get_openai_callback() as cb:
    result = algo_faq("Apakah program di Algoritma dapat diikuti orang awam?")
    print(f"Question: {result['query']}\nAnswer:\n{result['result']} \n")
    print("---- cb -----")
    print(cb)

Question: Apakah program di Algoritma dapat diikuti orang awam?
Answer:
 Ya, program di Algoritma dapat diikuti orang awam yang tidak memiliki latar belakang pendidikan IT sebelumnya. Kursus dasar-dasar data science, seperti dasar-dasar pemrograman dan statistik praktis, diajarkan sehingga sangat memungkinkan program ini terbuka untuk siapa saja yang tidak memiliki latar belakang IT. Contohnya seperti Reyna Cheryl Sondakh dan Samuel Gema. 

---- cb -----
Tokens Used: 2914
	Prompt Tokens: 2786
	Completion Tokens: 128
Successful Requests: 1
Total Cost (USD): $0.05828


In [94]:
with get_openai_callback() as cb2:
    result2 = algo_faq("Apakah program di Algoritma dapat diikuti orang awam?")
    print(f"Question: {result2['query']}\nAnswer:\n{result2['result']} \n")
    print("---- cb -----")
    print(cb2)

Question: Apakah program di Algoritma dapat diikuti orang awam?
Answer:
 Ya, program di Algoritma dapat diikuti orang awam yang tidak memiliki latar belakang pendidikan IT sebelumnya. Kursus dasar-dasar data science, seperti dasar-dasar pemrograman dan statistik praktis, diajarkan sehingga sangat memungkinkan program ini terbuka untuk siapa saja yang tidak memiliki latar belakang IT. Contohnya seperti Reyna Cheryl Sondakh dan Samuel Gema. 

---- cb -----
Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


After implementing the in-memory cache, the subsequent query showed a significant difference. The cache allowed the query to be completed without using any tokens, resulting in zero tokens used for both prompt and completion. There were no successful requests, and the total cost remained at $0.0. This demonstrates the effectiveness of the in-memory cache in avoiding redundant computations and minimizing token usage, leading to cost and time savings.

#### SQLite Cache

SQLite Cache is a type of caching mechanism used in LangChain to store and retrieve information efficiently. It utilizes an SQLite database, which is a lightweight, self-contained, and serverless database system.

In simple terms, think of the SQLite Cache as a small and fast storage space that LangChain uses to keep track of previously processed data. When you make a query or request, LangChain first checks if the answer to that request is already stored in the SQLite Cache. If it is, LangChain can quickly retrieve the answer from the cache without re-computing it, saving time and resources.

Using SQLite Cache helps to reduce repetitive computations, making the process faster and more efficient. It's like having a smart memory that remembers previous answers and provides them instantly when needed, without the need to redo the work every time. This way, LangChain can process requests more quickly and efficiently, improving the overall performance of the system.

To use SQLite cache in LangChain, we can implement it by defining `SQLiteCache(database_path=".langchain.db")` in `langchain.llm_cache`. This way, LangChain will store and retrieve cached data using an SQLite database located at the specified path (`.langchain.db` in this case). SQLite Cache helps improve efficiency by avoiding redundant computations and speeding up data retrieval, resulting in faster and more optimized performance for LangChain operations.

In [98]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [99]:
with get_openai_callback() as cb:
    result3 = algo_faq("Apakah ada alternatif pembayaran selain cash dan juga credit untuk menjadi student Algoritma?")
    print(f"Question: {result3['query']}\nAnswer:\n{result3['result']} \n")
    print("---- cb -----")
    print(cb)

Question: Apakah ada alternatif pembayaran selain cash dan juga credit untuk menjadi student Algoritma?
Answer:
 Ya, ada tiga mitra pembiayaan yang dapat menawarkan paket pembayaran hingga 18 bulan kepada siswa yang memenuhi syarat. Ketiga platform tersebut adalah Danacita, Edufund, Koinworks. 

---- cb -----
Tokens Used: 2865
	Prompt Tokens: 2795
	Completion Tokens: 70
Successful Requests: 1
Total Cost (USD): $0.0573


In [100]:
with get_openai_callback() as cb2:
    result4 = algo_faq("Apakah ada alternatif pembayaran selain cash dan juga credit untuk menjadi student Algoritma?")
    print(f"Question: {result4['query']}\nAnswer:\n{result4['result']} \n")
    print("---- cb -----")
    print(cb2)

Question: Apakah ada alternatif pembayaran selain cash dan juga credit untuk menjadi student Algoritma?
Answer:
 Ya, ada tiga mitra pembiayaan yang dapat menawarkan paket pembayaran hingga 18 bulan kepada siswa yang memenuhi syarat. Ketiga platform tersebut adalah Danacita, Edufund, Koinworks. 

---- cb -----
Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


After implementing SQLite cache in LangChain, the number of tokens used, prompt tokens, and completion tokens reduced to 0, indicating that no additional tokens were consumed and no new requests were made to the LLM provider. As a result, the total cost for the query became $0.0, and the SQLite cache efficiently retrieved the data without incurring any extra expenses. This demonstrates how caching can significantly optimize and economize repetitive queries, leading to improved performance and cost savings.

After understanding the implementation of in-memory cache and SQLite cache in LangChain, we can decide when to use each caching method based on the specific requirements of our application.

We can use in-memory cache when we have limited memory resources and want to store frequently accessed data temporarily. In-memory cache is ideal for scenarios where we need to quickly access and update the cached data, and the data can be discarded once it is no longer needed. Since the data is stored in memory, it allows for fast read and write operations, but it is not suitable for long-term storage or when the application is restarted, as the cache will be cleared.

On the other hand, we can use SQLite cache when we need to store larger amounts of data persistently and want to retain the cached data even after the application is restarted. SQLite cache is suitable for cases where we need a more permanent and reliable storage solution for cached data. It can handle larger datasets efficiently and provides the advantage of data persistence, allowing us to access the cached data across multiple sessions or restarts of the application.

In summary, the choice between in-memory cache and SQLite cache depends on the **specific use case and the trade-offs between memory usage, data persistence, and the need for fast read/write operations**. Both caching methods provide significant performance improvements and cost savings by reducing redundant API calls and token consumption when accessing the language model.

## Ethical Considerations


Generative AI powered by Large Language Models (LLMs) has brought remarkable advancements in natural language understanding and generation. As these models continue to evolve, it becomes crucial to address the ethical implications and potential societal impacts that come with their widespread use. In this article, we will delve into the ethical considerations and future implications of Generative AI with LLMs, shedding light on the responsible use of this technology.

### A Language for LLM Prompt Design: Guidance

Guidance prompt design in Large Language Models (LLMs) refers to the careful crafting of input texts or queries that effectively guide the model to produce desired and appropriate outputs. LLMs, such as GPT-3, can generate human-like text based on the context provided in the prompts. However, they are also susceptible to biases and can generate misleading or undesirable content if the prompts are not well-defined.

To design effective guidance prompts, several considerations should be taken into account:

1. **Clarity and Specificity**: The prompts should be clear, concise, and specific to the task or context. Avoid ambiguous language that may lead to uncertain or unintended responses.

2. **Context and Intent**: Provide sufficient context and specify the intended purpose of the prompt. Clearly state what information or response you are seeking from the model.

3. **Ethical and Inclusive Language**: Ensure that the prompts follow ethical guidelines and do not promote harmful content or biased perspectives. Use inclusive language and avoid any content that might be offensive or discriminatory.

4. **Fact-Checking and Verification**: Verify the information in the prompts before using the model's output. Fact-checking is essential to prevent the propagation of misinformation.

5. **Avoiding Leading Questions**: Avoid leading questions that might influence the model's response in a particular direction. Prompts should be neutral and unbiased.

6. **Sampling and Experimentation**: Experiment with different prompts and sample outputs to understand the model's behavior and choose the most suitable prompt.

7. **Handling Unknowns**: Consider how the model should respond to queries that are outside its trained knowledge. Provide a clear response strategy for unknown or out-of-context questions.

Guidance prompt design is crucial to harness the power of LLMs effectively and responsibly. It allows users to interact with the model in a controlled manner, ensuring that the generated content aligns with the intended use case and ethical standards. By providing clear and well-structured prompts, users can obtain accurate and contextually appropriate responses from LLMs while minimizing the risk of generating undesirable or harmful content.

Let's consider an example of a language model that can generate restaurant reviews based on given prompts.

**Good Prompt Design:**
Prompt: "Write a positive review for a vegetarian restaurant in the city center. Mention the ambiance, service, and your favorite dish."

Explanation: In this example, the prompt is clear and specific about the desired output, which is a positive review for a vegetarian restaurant. It provides context by mentioning the location (city center) and instructs the model to cover aspects like ambiance, service, and a favorite dish. This guidance ensures that the generated review will be relevant and aligned with the intent of praising a specific type of restaurant.

**Bad Prompt Design:**
Prompt: "Write a restaurant review."

Explanation: In this example, the prompt is very vague and lacks context. It does not specify the type of restaurant or any specific criteria for the review. As a result, the language model might generate a review for any random restaurant, making the output irrelevant and possibly nonsensical.

**Good Prompt Design:**
Prompt: "Write an informative article about the benefits of renewable energy sources. Include details on solar, wind, and hydro power."

Explanation: This prompt is well-designed as it provides clear instructions for the desired output, which is an informative article about renewable energy sources. It specifies the energy sources to be covered (solar, wind, and hydro power), ensuring that the generated article is focused and comprehensive.

**Bad Prompt Design:**
Prompt: "Energy."

Explanation: This prompt is very ambiguous and lacks clarity. It does not specify what type of information the language model should provide about energy. As a result, the output might be unrelated or scattered information about energy in general, making it less useful and coherent.

In summary, good prompt design is essential for guiding language models to produce relevant and accurate outputs. A well-crafted prompt should be clear, specific, and provide sufficient context to the model, while avoiding vagueness or leading questions that might result in irrelevant or biased content.

### Understanding the Ethical Considerations of Generative AI

Generative AI, including Large Language Models (LLMs), presents various ethical concerns that need careful consideration. One major concern is the possibility of generating harmful or misleading content, which can lead to negative impacts on individuals or society. Additionally, LLMs have the potential to amplify misinformation and spread false information at a large scale, creating challenges for maintaining accurate and reliable information in the digital space.

Another ethical issue is related to individual privacy, as LLMs can inadvertently process and generate sensitive or private information. This raises questions about data privacy and the need to implement robust mechanisms for data protection when using LLMs.

To address these ethical challenges, it is crucial for developers, researchers, and organizations to prioritize ethical guidelines and responsible practices when deploying LLMs in real-world applications. This includes implementing safeguards to prevent the generation of harmful or misleading content, promoting transparency in AI-generated content, and ensuring that LLMs are used for positive and beneficial purposes.

Moreover, it is essential to engage in continuous dialogue and collaboration with stakeholders to gather diverse perspectives on the ethical implications of LLM use. This can involve collaborating with ethicists, policymakers, and the wider public to develop frameworks and policies that promote ethical AI deployment and mitigate potential harms.

By being proactive in addressing ethical considerations, we can harness the potential of LLMs for positive advancements in various fields while ensuring that the deployment of AI aligns with societal values and ethical standards.

### Impact on Privacy, Bias, and Misinformation

The impressive capabilities of Large Language Models (LLMs) in generating human-like text also give rise to significant concerns regarding user privacy. When using LLMs to generate text, there is a risk of inadvertently revealing sensitive information. For instance, if an LLM is trained on datasets containing personal or confidential data, it may inadvertently generate content that discloses private information. To safeguard against such breaches, developers and organizations must implement rigorous data anonymization and privacy protection measures before utilizing LLMs in applications that involve sensitive data.

Moreover, LLMs can potentially perpetuate bias in their generated content. If the training data used to train the model contains biased information, the LLM may produce biased outputs, thus reinforcing existing social inequalities. This can have serious implications, especially in areas such as automated content generation, decision-making processes, or language translation, where biased outputs can perpetuate stereotypes and discrimination. Addressing bias in LLM-generated content requires careful selection and preprocessing of training data, as well as ongoing monitoring and evaluation of the model's outputs to ensure fairness and inclusivity.

Furthermore, the issue of misinformation is a critical aspect when using LLMs for content generation. While LLMs can produce coherent and plausible text, they may also generate false or misleading information. To combat misinformation, it is essential to incorporate robust fact-checking and verification mechanisms into the content generation process. Combining LLMs with human review and editorial oversight can help ensure the accuracy and reliability of the generated content.

> For example, in the context of news articles or social media posts, LLM-generated content should be verified by human journalists or fact-checkers before dissemination. This collaborative approach between AI and human expertise can enhance the credibility and trustworthiness of the information generated by LLMs.

In summary, addressing the ethical considerations of LLMs involves preserving user privacy, combating bias, and tackling misinformation. By implementing responsible practices, transparency, and human oversight, we can harness the full potential of LLMs while ensuring that their deployment aligns with ethical principles and societal values.

### Responsible Use of Large Language Models in Society

As Large Language Models (LLMs) become more widely available, it is crucial to promote their responsible use across different industries and applications. In this context, we will explore guidelines and best practices for deploying LLMs in various contexts, such as customer support, content generation, and chatbots.

Responsible use of LLMs involves several key aspects:

1. **Transparency**: Organizations and developers should be transparent about the use of LLMs and disclose when AI-generated content is involved. Users should be made aware that they might interact with AI systems and understand the limitations of the generated content.

2. **Ethical Content Generation**: LLMs have the potential to produce large amounts of content quickly. However, it is essential to ensure that the generated content adheres to ethical standards. This includes avoiding the dissemination of false or misleading information and being cautious not to generate harmful or offensive content.

3. **Bias Mitigation**: LLMs can inherit biases present in the training data. It is crucial to implement techniques that mitigate bias and promote fairness in the generated content. This can involve careful curation of training data, bias-aware fine-tuning, and post-processing to identify and correct biased outputs.

4. **Human Oversight**: While LLMs are powerful tools, they are not infallible. Integrating human oversight and review processes can help catch any errors, biases, or misleading information that the AI may produce. Human input can add context and verify the accuracy of the generated content.

**Example: Customer Support Chatbot**
Let's consider a customer support chatbot deployed by an e-commerce company. The chatbot uses LLMs to generate responses to customer queries. To ensure responsible use, the company follows the following practices:

- **Transparency**: At the beginning of the conversation, the chatbot clearly informs users that they are interacting with an AI-powered system. It also provides options for users to talk to a human representative if needed.

- **Ethical Content Generation**: The chatbot is programmed to avoid sharing any false or misleading information. It refrains from making promises it cannot keep and ensures that the generated responses are helpful and accurate.

- **Bias Mitigation**: The e-commerce company carefully curates the training data for the chatbot, making sure it represents a diverse range of customer queries and avoids biased content. Additionally, the chatbot is regularly monitored for any signs of biased outputs, and corrective actions are taken as needed.

- **Human Oversight**: While the chatbot handles many customer queries, it has a fallback mechanism where complex or sensitive inquiries are redirected to human customer support representatives. This ensures that critical issues are appropriately addressed by humans with the necessary expertise.

By following these responsible practices, the e-commerce company can leverage LLMs to enhance their customer support services while maintaining ethical and trustworthy interactions with their users.

### Discussion on the Future of Generative AI and Its Potential Impact

The future implications of Generative AI with LLMs hold both promise and challenges. We will explore potential positive outcomes, such as enhancing communication, creativity, and problem-solving, as well as the concerns surrounding the misuse of this technology. Engaging in an informed discussion on the future impact of LLMs will help us shape policies and guidelines that balance innovation with ethical considerations.

> **Example Scenario:**
Consider a scenario where a chatbot powered by a large language model is deployed by a company to handle customer queries and complaints. In this context, the responsible use of LLMs would involve carefully designing prompts to avoid generating misleading or harmful responses. The company would implement strict privacy protocols to ensure that sensitive customer data remains confidential and is not leaked through the chatbot's responses. Additionally, the chatbot's content would be regularly monitored and fact-checked to prevent the dissemination of misinformation. By adhering to these ethical considerations, the company can enhance customer support while safeguarding user privacy and trust.

In conclusion, the article will emphasize the need for ongoing research, open dialogues, and ethical frameworks to navigate the transformative potential of Generative AI with Large Language Models responsibly. Addressing ethical considerations and understanding future implications will help us harness the benefits of this technology while mitigating potential risks for a more equitable and ethically sound AI-driven future.

# Summary



In this module, we have explored various aspects of Leveraging Large Language Models (LLM) for Web Scraping and Information Extraction. We learned about optimizing LLMs to improve efficiency and cost-effectiveness. Additionally, we discussed the ethical considerations and future implications of using generative AI with large language models.

With the knowledge gained from this module, we can harness the power of LLMs for web scraping and information extraction to extract valuable insights from unstructured data sources. By optimizing LLMs, we can enhance the performance and reduce the computational resources required for processing large volumes of text data. Moreover, we have deepened our understanding of the ethical challenges associated with using generative AI and the responsibilities of developers and organizations to prioritize ethical guidelines when deploying LLMs in real-world applications.

By combining web scraping, information extraction, and ethical considerations, we can create responsible and powerful AI systems that deliver accurate and relevant information while upholding ethical standards. This knowledge empowers us to develop applications and solutions that leverage LLMs to solve complex problems in various domains, such as natural language processing, chatbots, sentiment analysis, question-answering systems, and more. As we continue to advance our understanding of LLMs and their capabilities, we can explore new opportunities to apply these technologies responsibly and ethically, shaping the future of AI-powered applications and services.